In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from pathlib import Path
from tqdm import tqdm
import polars as pl
from datetime import datetime
gpd.options.use_pygeos = True

/tmp/ipykernel_448828/4015039897.py:9: UserWarning: pygeos support was removed in 1.0. geopandas.use_pygeos is a no-op and will be removed in geopandas 1.1.
  gpd.options.use_pygeos = True


In [2]:
print("hello")

hello


In [3]:
# Helper function for timestamped print statements
def log(message):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}", flush=True)

In [4]:
# # This code generates duplicate rows for the grids that intersect with different geology polygons
i = 24
slope_crs = "EPSG:4326"  # Assuming slope data in WGS 84 CRS

slope_data_rect = f"/mnt/c/Users/melis/Desktop/add_slope_parquet/10_07_25_gdf_parquet_version_repaired_serial/grid_numbers_added/grid_numbers_fixed/gdf_rect{i}_slope_inside.parquet"
log("Starting to read slope file...")

#slope_data_rect1 = pl.read_parquet(slope_data_rect1, columns=["X_max", "X_min", "Y_max", "Y_min"])
slope_data_rect = pl.read_parquet(slope_data_rect, columns=["X_max", "X_min", "Y_max", "Y_min", "Slope Value"])
log("Slope file read successfully.")

[2025-10-14 13:06:27] Starting to read slope file...
[2025-10-14 13:06:27] Slope file read successfully.


In [5]:
log("Converting slope data from Polars to Pandas...")
slope_data_rect = slope_data_rect.to_pandas()
log("Conversion from Polars to Pandas completed.")

[2025-10-14 13:06:27] Converting slope data from Polars to Pandas...
[2025-10-14 13:06:27] Conversion from Polars to Pandas completed.


In [6]:
slope_data_rect.head()

,X_max,X_min,Y_max,Y_min,Slope Value
0,-115.855741,-115.855833,35.970370,35.970278,0.586749
1,-115.855926,-115.856018,35.970278,35.970185,0.596100
2,-115.854907,-115.855000,35.970278,35.970185,0.481124
3,-115.855741,-115.855833,35.970556,35.970463,0.611770
4,-115.855926,-115.856018,35.970370,35.970278,0.593905


In [7]:
# === Load geology polygons ===
slope_crs = "EPSG:4326" 
log("Starting to read geology file...")
geology_gdf = gpd.read_file(r"/mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geology/shapefiles_Geology/shapefiles_Geology/GMC_geo_poly.shp")
if geology_gdf.crs is None or geology_gdf.crs.to_string() != slope_crs:
    log("Reprojecting geology shapefile to match slope CRS...")
    geology_gdf = geology_gdf.to_crs(slope_crs)
log("Geology file read and CRS verified.")

[2025-10-14 13:06:27] Starting to read geology file...
[2025-10-14 13:06:29] Reprojecting geology shapefile to match slope CRS...
[2025-10-14 13:06:29] Geology file read and CRS verified.


In [8]:
# import multiprocessing as mp
# from functools import partial
# import gc
# import pyarrow.parquet as pq

# # Helper function to process a single chunk (must be defined at module level for multiprocessing)
# def process_chunk(chunk, slope_crs):
#     return gpd.GeoDataFrame(
#         chunk,
#         geometry=[
#             box(xmin, ymin, xmax, ymax)
#             for xmin, ymin, xmax, ymax in zip(
#                 chunk["X_min"], chunk["Y_min"], chunk["X_max"], chunk["Y_max"]
#             )
#         ],
#         crs=slope_crs
#     )

# # This code generates duplicate rows for the grids that intersect with different geology polygons
# i = 3
# slope_crs = "EPSG:4326"  # Assuming slope data in WGS 84 CRS

# slope_data_path = f"/mnt/c/Users/melis/Desktop/add_slope_parquet/10_07_25_gdf_parquet_version_repaired_serial/grid_numbers_added/grid_numbers_fixed/gdf_rect{i}_slope_inside.parquet"
# log("Starting to read slope file in chunks...")

# # Read parquet metadata to get total rows
# parquet_file = pq.ParquetFile(slope_data_path)
# total_rows = parquet_file.metadata.num_rows
# log(f"Total rows in parquet file: {total_rows}")

# chunk_size = 200000  # Adjust based on available memory
# num_processes = mp.cpu_count() // 7  # Use half your CPU cores (e.g., 4-8); adjust as needed
# slope_gdfs = []

# # Calculate total number of chunks for tqdm
# total_chunks = (total_rows + chunk_size - 1) // chunk_size

# log("Processing slope data in parallel chunks...")

# # Generator to read parquet file in chunks
# def chunk_generator(parquet_path, chunk_size, columns):
#     parquet_file = pq.ParquetFile(parquet_path)
#     for batch in parquet_file.iter_batches(batch_size=chunk_size, columns=columns):
#         df_chunk = batch.to_pandas()
#         # Downcast numeric columns
#         df_chunk["X_min"] = pd.to_numeric(df_chunk["X_min"], downcast="float")
#         df_chunk["Y_min"] = pd.to_numeric(df_chunk["Y_min"], downcast="float")
#         df_chunk["X_max"] = pd.to_numeric(df_chunk["X_max"], downcast="float")
#         df_chunk["Y_max"] = pd.to_numeric(df_chunk["Y_max"], downcast="float")
#         yield df_chunk

# # Process chunks in parallel with progress tracking
# with mp.Pool(processes=num_processes) as pool:
#     slope_gdfs = list(tqdm(
#         pool.imap(
#             partial(process_chunk, slope_crs=slope_crs),
#             chunk_generator(slope_data_path, chunk_size, ["X_max", "X_min", "Y_max", "Y_min", "Slope Value"])
#         ),
#         total=total_chunks,
#         desc="Processing chunks"
#     ))

# slope_gdf = pd.concat(slope_gdfs, ignore_index=True)
# log("Slope grids processed in parallel chunks and combined.")

# # Free memory after concatenation
# del slope_gdfs
# gc.collect()

In [9]:
slope_data_rect["X_min"] = pd.to_numeric(slope_data_rect["X_min"], downcast="float")
slope_data_rect["Y_min"] = pd.to_numeric(slope_data_rect["Y_min"], downcast="float")
slope_data_rect["X_max"] = pd.to_numeric(slope_data_rect["X_max"], downcast="float")
slope_data_rect["Y_max"] = pd.to_numeric(slope_data_rect["Y_max"], downcast="float")

In [10]:
# del slope_gdfs
# import gc

# gc.collect()

In [11]:
log("Converting slope grids into polygons...")
slope_gdf = gpd.GeoDataFrame(
    slope_data_rect,
    geometry=[
        box(xmin, ymin, xmax, ymax)
        for xmin, ymin, xmax, ymax in zip(
            slope_data_rect["X_min"],
            slope_data_rect["Y_min"],
            slope_data_rect["X_max"],
            slope_data_rect["Y_max"],
        )
    ],
    crs=slope_crs
)
log("Slope grids converted into polygons.")

# import multiprocessing as mp
# from functools import partial
# import gc  # Add this if not already present

# # Helper function to process a single chunk (must be defined at module level for multiprocessing)
# def process_chunk(chunk, slope_crs):
#     return gpd.GeoDataFrame(
#         chunk,
#         geometry=[
#             box(xmin, ymin, xmax, ymax)
#             for xmin, ymin, xmax, ymax in zip(
#                 chunk["X_min"], chunk["Y_min"], chunk["X_max"], chunk["Y_max"]
#             )
#         ],
#         crs=slope_crs
#     )

# log("Processing slope_data_rect in parallel chunks...")
# chunk_size = 200000  # Adjust based on available memory
# num_processes = mp.cpu_count() // 6  # Use half your CPU cores (e.g., 4-8); adjust as needed

# # Split data into chunks
# chunks = [slope_data_rect.iloc[start:start + chunk_size] for start in range(0, len(slope_data_rect), chunk_size)]

# # Process chunks in parallel with progress tracking
# with mp.Pool(processes=num_processes) as pool:
#     slope_gdfs = list(tqdm(
#         pool.imap(partial(process_chunk, slope_crs=slope_crs), chunks),
#         total=len(chunks),
#         desc="Processing chunks"
#     ))

# slope_gdf = pd.concat(slope_gdfs, ignore_index=True)
# log("Slope grids processed in parallel chunks and combined.")

# # Free memory after concatenation
# del slope_gdfs
# gc.collect()

[2025-10-14 13:06:30] Converting slope grids into polygons...
[2025-10-14 13:07:16] Slope grids converted into polygons.


In [12]:
# === Step 1: Candidate matches ===
log("Performing spatial join between slope grids and geology polygons...")
tqdm.pandas()  # Enable tqdm for pandas operations

candidates = gpd.sjoin(
    slope_gdf.progress_apply(lambda x: x),  # Wrap slope_gdf with tqdm for progress tracking
    geology_gdf[["geometry", "PTYPE"]],
    how="left",
    predicate="intersects"
)
log("Spatial join completed.")

[2025-10-14 13:07:16] Performing spatial join between slope grids and geology polygons...


100%|██████████| 6/6 [00:00<00:00, 30.63it/s]


[2025-10-14 13:24:40] Spatial join completed.


In [13]:
candidates.head()

,X_max,X_min,Y_max,Y_min,Slope Value,geometry,index_right,PTYPE
0,-115.855743,-115.855835,35.970371,35.970276,0.586749,"POLYGON ((-115.85574 35.97028, -115.85574 35.9...",13906.0,Q
1,-115.855927,-115.856018,35.970276,35.970184,0.596100,"POLYGON ((-115.85593 35.97018, -115.85593 35.9...",13906.0,Q
2,-115.854904,-115.855003,35.970276,35.970184,0.481124,"POLYGON ((-115.8549 35.97018, -115.8549 35.970...",13906.0,Q
3,-115.855743,-115.855835,35.970554,35.970463,0.611770,"POLYGON ((-115.85574 35.97046, -115.85574 35.9...",13906.0,Q
4,-115.855927,-115.856018,35.970371,35.970276,0.593905,"POLYGON ((-115.85593 35.97028, -115.85593 35.9...",13906.0,Q


In [14]:
# Check for missing values in the PTYPE column
missing_ptype_count = candidates["PTYPE"].isna().sum()
log(f"Number of missing values in PTYPE column: {missing_ptype_count}")

# Get the total length of the candidates DataFrame
total_candidates_length = len(candidates)
log(f"Total number of rows in candidates DataFrame: {total_candidates_length}")

total_initial_length = len(slope_data_rect)
log(f"Total number of rows in initial DataFrame: {total_initial_length}")


[2025-10-14 13:24:40] Number of missing values in PTYPE column: 234
[2025-10-14 13:24:40] Total number of rows in candidates DataFrame: 5992227
[2025-10-14 13:24:40] Total number of rows in initial DataFrame: 5964689


In [15]:
log("Identifying grids with duplicates...")
dupes = candidates[candidates.duplicated(keep=False)]  # Check for duplicates across all columns

log(f"Total slope grids: {len(slope_gdf)}")
log(f"Grids with multiple geology overlaps: {dupes.index.nunique()}")


[2025-10-14 13:24:40] Identifying grids with duplicates...
[2025-10-14 13:24:49] Total slope grids: 5964689
[2025-10-14 13:24:49] Grids with multiple geology overlaps: 0


In [16]:
# Count occurrences of each slope grid in the spatial join
duplicate_counts = candidates.index.value_counts()

# Number of slope grids with multiple matches
num_multiple_matches = (duplicate_counts > 1).sum()
log(f"Number of slope grids with multiple geology overlaps: {num_multiple_matches}")

# Maximum number of matches for a single slope grid
max_matches = duplicate_counts.max()
log(f"Maximum number of matches for a single slope grid: {max_matches}")

[2025-10-14 13:24:49] Number of slope grids with multiple geology overlaps: 27497
[2025-10-14 13:24:49] Maximum number of matches for a single slope grid: 3


In [17]:
# Check for unmatched slope grids (no geology intersection)
log("Checking for unmatched slope grids...")

# Count rows where index_right is NaN (no geology match)
unmatched_count = candidates["index_right"].isna().sum()
total_grids = len(candidates)
matched_count = total_grids - unmatched_count

log(f"Total slope grids: {total_grids}")
log(f"Matched with geology: {matched_count}")
log(f"Unmatched (no geology): {unmatched_count}")
log(f"Match rate: {(matched_count / total_grids) * 100:.2f}%")

# Alternative: Check using PTYPE column
unmatched_ptype = candidates["PTYPE"].isna().sum()
log(f"Unmatched via PTYPE check: {unmatched_ptype}")

# Show a few examples of unmatched grids
if unmatched_count > 0:
    log("Sample of unmatched grids:")
    unmatched_grids = candidates[candidates["index_right"].isna()][["X_min", "Y_min", "X_max", "Y_max", "Slope Value", "PTYPE"]].head(10)
    print(unmatched_grids)
else:
    log("All slope grids have geology matches!")

[2025-10-14 13:24:49] Checking for unmatched slope grids...
[2025-10-14 13:24:49] Total slope grids: 5992227
[2025-10-14 13:24:49] Matched with geology: 5991993
[2025-10-14 13:24:49] Unmatched (no geology): 234
[2025-10-14 13:24:49] Match rate: 100.00%
[2025-10-14 13:24:49] Unmatched via PTYPE check: 234
[2025-10-14 13:24:49] Sample of unmatched grids:
           X_min      Y_min       X_max      Y_max  Slope Value PTYPE
99   -115.853798  35.969353 -115.853706  35.969444     0.566359   NaN
286  -115.853333  35.968983 -115.853241  35.969074     0.598253   NaN
385  -115.852966  35.968704 -115.852867  35.968796     0.516573   NaN
414  -115.856293  35.971294 -115.856201  35.971390     0.640616   NaN
467  -115.854164  35.969631 -115.854073  35.969723     0.589915   NaN
514  -115.854630  35.970001 -115.854538  35.970093     0.581559   NaN
695  -115.850464  35.966759 -115.850372  35.966850     0.532298   NaN
826  -115.850830  35.967037 -115.850739  35.967129     0.494258   NaN
1108 -115.85129

In [ ]:
log("Calculating intersection areas for each row in candidates...")

if unmatched_count==0:
    # Add a new column for intersection area
    candidates["intersection_area"] = candidates.apply(
        lambda row: row.geometry.intersection(
            geology_gdf.loc[row.index_right, "geometry"]
        ).area if row.index_right is not None else 0,
        axis=1
    )

    log("Intersection areas calculated successfully.")

    # Check the first few rows to verify
    log("Preview of candidates with intersection areas:")
    print(candidates.head())

else:
    print("There are unmatched geology")
    # Add a new column for intersection area
    candidates["intersection_area"] = candidates.apply(
    lambda row: row.geometry.intersection(
        geology_gdf.loc[row.index_right, "geometry"]
    ).area if pd.notna(row.index_right) else 0,  # ✅ Use pd.notna() instead
    axis=1
)

    log("Intersection areas calculated successfully.")

    # Check the first few rows to verify
    log("Preview of candidates with intersection areas:")
    print(candidates.head())


[2025-10-14 13:34:22] Calculating intersection areas for each row in candidates...
There are unmatched geology


In [ ]:
log("Resolving duplicates by keeping the largest overlap...")

# Sort by overlap area in descending order and drop duplicates based on slope grid index
candidates_resolved = candidates.sort_values("intersection_area", ascending=False).drop_duplicates(subset=["geometry"])

log(f"Number of rows after resolving duplicates: {len(candidates_resolved)}")

# Combine resolved duplicates with single-match grids
# log("Combining resolved duplicates with single-match grids...")
# unique_matches = candidates[~candidates.index.duplicated(keep=False)]  # grids with exactly 1 geology
# slope_with_geo = pd.concat([unique_matches, candidates_resolved], ignore_index=True)

# log(f"Total number of rows after combining: {len(slope_with_geo)}")

# # Clean the final GeoDataFrame
# log("Cleaning final GeoDataFrame...")
# slope_with_geo = slope_with_geo.drop(columns=["index_right", "intersection_area"], errors="ignore")
# log("Final GeoDataFrame prepared.")

# # Save the final GeoDataFrame to a Parquet file
# log("Saving the final GeoDataFrame to a Parquet file...")
# output_path = Path("/mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geolog/slope_geology_grid_number_file/gdf_rect1_slope_geo_gridno.parquet")
# slope_with_geo.to_parquet(output_path, index=False)
# log(f"Final GeoDataFrame saved to {output_path}.")

[2025-10-13 19:24:30] Resolving duplicates by keeping the largest overlap...
[2025-10-13 19:28:38] Number of rows after resolving duplicates: 56082601


In [ ]:
#Sanity checks

assert len(candidates_resolved) == len(slope_gdf)

#are there any duplicates?
dupes = candidates_resolved.duplicated(subset=["X_min","Y_min","X_max","Y_max"]).sum()
print("Duplicate slope grids after resolution:", dupes)


#how many missing geology/ptype I have
missing = candidates_resolved["PTYPE"].isna().sum()
print("Grids with missing geology:", missing)

#coverage ratio

coverage_ratio = 1 - (missing / len(candidates_resolved))
print("Coverage ratio:", coverage_ratio)






Duplicate slope grids after resolution: 0
Grids with missing geology: 0
Coverage ratio: 1.0


In [ ]:
#are the grid numbers still correct?



In [ ]:
candidates_resolved.head()

,X_max,X_min,Y_max,Y_min,Slope Value,geometry,index_right,PTYPE,intersection_area
56082577,-118.427589,-118.428520,32.804352,32.803425,6.853119,"POLYGON ((-118.42759 32.80342, -118.42759 32.8...",13898,water,8.628122e-07
56082571,-118.425369,-118.426300,32.804539,32.803612,2.799100,"POLYGON ((-118.42537 32.80361, -118.42537 32.8...",10948,Tv,8.628122e-07
56082570,-118.425552,-118.426483,32.804722,32.803795,2.435479,"POLYGON ((-118.42555 32.80379, -118.42555 32.8...",10948,Tv,8.628122e-07
56082567,-118.422867,-118.423798,32.805092,32.804165,3.283050,"POLYGON ((-118.42287 32.80416, -118.42287 32.8...",13898,water,8.628122e-07
30,-117.674812,-117.675743,34.108425,34.107498,4.900031,"POLYGON ((-117.67481 34.1075, -117.67481 34.10...",13871,Q,8.628122e-07


In [ ]:
#drop the geometry and indec_right and intersection area
#save to "C:\Users\melis\Desktop\10_09_geologyand_landslide\10_09_25_geology\slope_geology"

log("Dropping unnecessary columns before saving...")
candidates_resolved_cleaned = candidates_resolved.drop(columns=["geometry", "index_right", "intersection_area"], errors="ignore")

log("Saving the cleaned DataFrame to the specified directory...")
output_path = Path(f"/mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geology/slope_geology_10_10_25/gdf_rect{i}_slope_geology_nogridnumber.parquet")
candidates_resolved_cleaned.to_parquet(output_path, index=False)

log(f"Cleaned DataFrame saved to {output_path}.")

[2025-10-13 19:38:14] Dropping unnecessary columns before saving...
[2025-10-13 19:38:15] Saving the cleaned DataFrame to the specified directory...
[2025-10-13 19:38:24] Cleaned DataFrame saved to /mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geology/slope_geology_10_10_25/gdf_rect12_slope_geology_nogridnumber.parquet.


In [ ]:
#read_file head
rect = pd.read_parquet(fr"/mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geology/slope_geology_10_10_25/gdf_rect{i}_slope_geology_nogridnumber.parquet")
print(rect.head())
#print how many rows
print(len(rect))
#compare size with slope only no grid number

        X_max       X_min      Y_max      Y_min  Slope Value  PTYPE
0 -118.427589 -118.428520  32.804352  32.803425     6.853119  water
1 -118.425369 -118.426300  32.804539  32.803612     2.799100     Tv
2 -118.425552 -118.426483  32.804722  32.803795     2.435479     Tv
3 -118.422867 -118.423798  32.805092  32.804165     3.283050  water
4 -117.674812 -117.675743  34.108425  34.107498     4.900031      Q
56082601


In [ ]:
#add grid numbers
#after all processing for all rects are done

# import pandas as pd
# import numpy as np
# from pathlib import Path
# import gc

# # === CONFIG ===
# input_dir = Path("/mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geology/slope_geology_10_10_25")

# output_dir =  Path("/mnt/c/Users/melis/Desktop/10_09_geologyand_landslide/10_09_25_geology/slope_geology_grid_number_10_10_25")
# output_dir.mkdir(exist_ok=True)

# # === Collect parquet files ===
# files = sorted(input_dir.glob("*.parquet"), key=lambda f: int("".join(filter(str.isdigit, f.stem))))

# # === Recalculate Grid Numbers ===
# current_start = 1  # Start GridNumber from 1
# for file in files:
#     print(f"Processing: {file.name}", flush=True)
#     try:
#         # Read the file
#         df = pd.read_parquet(file)

#         # Recalculate GridNumber
#         nrows = len(df)
#         df["GridNumber"] = np.arange(current_start, current_start + nrows, dtype=np.int64)

#         # Save the updated file
#         out_path = output_dir / file.name
#         df.to_parquet(out_path, index=False)

#         # Update the starting index for the next file
#         current_start += nrows
#         del df
        

#         print(f"✅ Fixed GridNumber for: {file.name} ({nrows} rows)", flush=True)

#         gc.collect()
#     except Exception as e:
#         print(f"❌ Failed to process {file.name}: {e}", flush=True